In [31]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split


In [32]:
star1=pd.read_csv(r"C:\Users\spand\Desktop\Restuarnt\one-star-michelin-restaurants.csv")
star2=pd.read_csv(r"C:\Users\spand\Desktop\Restuarnt\two-stars-michelin-restaurants.csv")
star3=pd.read_csv(r"C:\Users\spand\Desktop\Restuarnt\three-stars-michelin-restaurants.csv")


In [120]:
res_df=pd.concat([star1,star2,star3],ignore_index=True)


In [121]:
res_df.drop(columns=['zipCode'], inplace=True)
res_df['city'].fillna("Unknown", inplace=True)
res_df['price'].fillna("Unknown", inplace=True)
res_df['name'] = res_df['name'].str.lower().str.strip()
res_df['cuisine'] = res_df['cuisine'].str.lower().str.strip()
print(res_df.isnull().sum())

name         0
year         0
latitude     0
longitude    0
city         0
region       0
cuisine      0
price        0
url          0
dtype: int64


C:\Users\spand\AppData\Local\Temp\ipykernel_9916\136146628.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  res_df['city'].fillna("Unknown", inplace=True)
C:\Users\spand\AppData\Local\Temp\ipykernel_9916\136146628.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


vectorizer = TfidfVectorizer(stop_words='english')
cuisine_matrix = vectorizer.fit_transform(res_df['cuisine'])


cosine_sim = cosine_similarity(cuisine_matrix)


similarity_df = pd.DataFrame(cosine_sim, index=res_df['name'], columns=res_df['name'])


In [123]:
from difflib import get_close_matches

def find_closest_match(restaurant_name):
    matches = get_close_matches(restaurant_name.lower().strip(), similarity_df.index, n=1, cutoff=0.6)
    return matches[0] if matches else None


In [124]:
def recommend_restaurant(restaurant_name, top_n=5):
    restaurant_name = find_closest_match(restaurant_name) 
    
    if not restaurant_name or restaurant_name not in similarity_df.index:
        return f"Restaurant '{restaurant_name}' not found! Try another name."

    recommendations = similarity_df[restaurant_name].sort_values(ascending=False).iloc[1:top_n+1]
    return recommendations.index.tolist()


In [125]:
from fuzzywuzzy import process

def find_closest_match(restaurant_name):
   
    closest_match = process.extractOne(restaurant_name.lower().strip(), similarity_df.index)
    return closest_match[0] if closest_match else None

def recommend_restaurant(restaurant_name, top_n=5):
    restaurant_name = find_closest_match(restaurant_name)  
    
    if restaurant_name not in similarity_df.index:
        return f"Restaurant '{restaurant_name}' not found!."

    recommendations = similarity_df[restaurant_name].sort_values(ascending=False).iloc[1:top_n+1]
    return recommendations.index.tolist()


In [126]:
restaurant_name = input("Enter a restaurant name: ").strip().lower()
recommendations = recommend_restaurant(restaurant_name, top_n=5)

if isinstance(recommendations, list):
    print("Recommended restaurants:")
    for res in recommendations:
        print(res)
else:
    print(recommendations)  


Enter a restaurant name:  sunny


Recommended restaurants:
shin sushi
mori sushi
urasawa
shunji
nozawa bar
